## ⚠️ <font color=red>**Notebook is not guaranteed to work.**</font>

---

## Example 🖼️
| Thumbnail | Video (*± Middle Frame*) |
|:---:|:---:|
| <img src="https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Thumbnail.png" height=200> | <img src= "https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Frame.png" height=200> |
| *Basing on the selected number of colors in the palletized<br>image, a 16:9 background is generated, onto the<br>center of which the image is placed.* | *Generates the video with dimensions that<br>match the input image, and appends<br>the specified audio to it.*|

---
<br>

### 📱 **Information for mobile users**
<font color="red"><b>If cells don't output anything and are finished immediately, please re-run the site in the <code>Desktop site</code> mode</b></font> (Chrome)

<br>

---
<br>

[![](https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge)](https://github.com/kubinka0505/YTMVG)

# Main ⚙️

In [ ]:
#@title # <font color=lime>**1.**</font> Get and setup the required code ⚙️
#@markdown ---
#@markdown #@a
#@markdown
#@markdown Clone
#@markdown - [**YTMVG**](https://github.com/kubinka0505/YTMVG)
#@markdown
#@markdown Install
#@markdown - Packages
#@markdown	  - [JPEGOptim](https://github.com/tjko/jpegoptim)&emsp;&emsp;&emsp;&emsp;Thumbnail optimization 📉
#@markdown - Modules
#@markdown	  - [STY](https://github.com/feluxe/sty)&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Colored output 🎨
#@markdown	  - [FFPB](https://github.com/kubinka0505/ffpb-fix)&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; [*FFmpeg*](https://github.com/FFmpeg/FFmpeg) progress bar ⌛
#@markdown	  - [Mutagen](https://github.com/quodlibet/mutagen)&emsp;&emsp;&emsp;&emsp;&emsp; Video length handler ⚙️

from os import *
from PIL import Image
from time import time
from pathlib import Path
from shutil import rmtree
from zipfile import ZipFile
from base64 import b64encode
from requests import Session
from google.colab import files
from PIL.ImageColor import getrgb as RGB
from IPython.core.display import display, HTML
del open
chdir("/content")

#-=-=-=-#

!rm -rf YTMVG
!git clone https://github.com/kubinka0505/YTMVG
chdir("YTMVG")
!apt-get install -qq -o=Dpkg::Use-Pty=0 jpegoptim
%pip install -r requirements.txt | grep -v "already satisfied"

#-=-=-=-#

from sty import fg
from colour import Color

session = Session()
session.headers.update(
	{
		"User-Agent":
		"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
	}
)
get = session.get

class Styles:
	"""Colored Prints."""
	OK = "#0C0"
	Info = "#90F"
	Warning = "#FC0"

Class = Styles
for Variable in list(vars(Class))[2:-2]:
	exec('{0}.{1} = "{2}"'.format(
		Class.__name__, Variable,
		fg(*RGB(getattr(Class, Variable)))
		)
	)

Styles.Reset = fg.rs

In [ ]:
#@title # <font color=lime>**2.**</font> Upload Files 📤

#@markdown ---

#@markdown ## **Direct** Media `URL`s
Cover = "" #@param {type: "string"}
Audio = "" #@param {type: "string"}

#@markdown ---

#@markdown ℹ️ **Information**
#@markdown - Leave blank for manual upload.
#@markdown - **Files in URL fields are processed faster.** (stream)
#@markdown - <font color=red>**Detecting cover art is not supported.**</font>
#@markdown - **Links are [normalized](https://wikipedia.org/wiki/URI_normalization).**

#@markdown ⚠️ **Warnings**
#@markdown - URLs must start with **internet** [transfer protocols](https://wikipedia.org/wiki/Comparison_of_file_transfer_protocols#Overview).
#@markdown - <font color=red>**`Cover` image cannot be extensionless!**</font>
#@markdown - <font color=red>**In case of manual interruption, <u>all possibly previously uploaded files are ignored!**</u></font>

#-=-=-=-#

try:
	from mutagen import File
except:
	raise SystemExit("Modules not found!\nPlease run the first cell.")

chdir("/content")
!rm * > /dev/null 2>&1

#-=-=-=-#

if all((Cover, Audio)):
	State = "fetched"
elif any((Cover, Audio)):
	State = "fetched/uploaded"
elif not all((Cover, Audio)):
	State = "uploaded"

#-=-=-=-#

def Upload_Media(File_: str, Titles: list, Method: type(abs)):
	"""Custom handler for files uploading"""
	#-=-=-=-#
	Titles[0] = Titles[0].title()
	Titles[1] = Titles[1].lower()
	#-=-=-=-#
	if not File_:
		print("\nUpload {1}{0}{2}...".format(Titles[0], Styles.Warning, Styles.Reset))
		File_ = files.upload()
		File_ = list(File_.keys())[0]
	else:
		File_ = get(File_, stream = True)
		if File_.ok:
			File_Name = File_.url.split("/")[-1].split("#")[0].split("?")[0].split("&")[0]
			open(File_Name, "wb").write(File_.content)
			File_ = File_Name
		else:
			raise SystemExit("{0} returns status code {1} ({2})".format(Titles[0], File_.status_code, File_.reason.title()))
	File_ = path.abspath(File_)
	try:
		Method(File_)
	except FileNotFoundError:
		pass
	except:
		raise OSError("Not an {0} file!".format(Titles[1]))
	return File_


#-=-=-=-#

print("Uploading Media...")

try:
	Cover = Upload_Media(Cover, ["Cover", "Image"], Image.open)
	Audio = Upload_Media(Audio, ["Audio",] * 2, File)
except (IndexError, KeyboardInterrupt):
	try:
		remove(Cover)
		del Cover
	except:
		pass
	#-=-=-=-#
	try:
		remove(Audio)
		del Audio
	except:
		pass
	#-=-=-=-#
	raise SystemExit("Files uploading aborted")

print("""
	{2}Cover{3}\t{0}
	{2}Audio{3}\t{1}{4}
	""".format(
		Cover.split(sep)[-1], Audio.split(sep)[-1],
		Styles.Warning, Styles.Info, Styles.Reset
	)[:-1]
)

print("{1}Files {0} successfully!{2}".format(State, Styles.OK, Styles.Reset))

In [ ]:
try:
	chdir("/content/YTMVG")
except:
	raise SystemExit("Modules not found!\nPlease run the first cell.")

#-=-=-=-#

__name__	= get("http://172.28.0.2:9000/api/sessions").json()[0]["name"]
__author__	= "kubinka0505"
__credits__	= __author__
__version__	= "1.0"
__date__		= "28.04.2022"
__status__	= "Mature"
__license__	= "GPLv3"

#-=-=-=-#
#@title # <font color=lime>**3.**</font> Fill the settings data form <font color=orange>**and then**</font> run
#@markdown ### ⚠️ <font color=red>**The `Videos` directory is cleared on this cell start!**</cont>
#@markdown ---

#@markdown # Thumbnail *(background)* 🖼️
Palette_Size = 10 #@param {type: "slider", min: 8, max: 16, step: 1}
Cluster = "Dominant" #@param ["Dominant", "Average", "Alternative"]
Brightness = 50 #@param {type: "slider", min: 25, max: 100, step: 1}

#@markdown ---

#@markdown # Video 📹
Maximum_Resolution = "1080p" #@param ["4320p", "2160p", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"]
#@markdown # Audio 🔊
Bitrate = 265 #@param {type: "slider", min: 96, max: 265, step: 1}
Lossless = False #@param {type: "boolean"}

#@markdown ---

#@markdown # Miscellaneous ⚙️
Show_Command = False #@param {type: "boolean"}
Show_Help = False #@param {type: "boolean"}

#-=-=-=-#

# Files Existence Check
for File_ in ["Cover", "Audio"]:
	if not File_ in locals():
		raise FileNotFoundError(File_ + " was not uploaded!\nPlease re-run the previous cell.")

#-=-=-=-=-=-#

# Environment Variables Assignment
environ["COVER"] = Cover
environ["AUDIO"] = Audio
environ["CLUSTER"] = Cluster
environ["PALETTE"] = Palette_Size
environ["BRIGHTNESS"] = str(float(Brightness))
environ["RESOLUTION"] = Maximum_Resolution.rstrip("p")
environ["BITRATE"] =  str(0 if Lossless else Bitrate)

if Show_Debug:
	print("\r", end = Styles.Warning)
	environ["DEBUG"] = "echo " + environ["DEBUG"]
else:
	!rm -rf "Videos"
	!mkdir "Videos"
	environ["DEBUG"] = "python"

try:
	!$DEBUG "YTMVG.py" \
		-i $COVER -a $AUDIO \
		-bgs $PALETTE -c $CLUSTER -bgb $BRIGHTNESS\%\
		-ab $BITRATE\k -r $RESOLUTION\p -nod

	if all((Debug, Show_Help)):
		print(Styles.Reset + "\n\r" + "─" * 33)
		!echo & python "YTMVG.py" -h
	
	if Debug:
		print(Styles.Reset + "\n\r" + "─" * 33 + "\n")
		raise SystemExit("Debug commands processed, exiting.")

	print("\r", end = Styles.Reset)
except KeyboardInterrupt:
	raise SystemExit("YTMVG was interrupted by the user, exiting")

#-=-=-=-=-=-#

# Getting Files
Videos = "Videos"
Directory = [path.join(Videos, File) for File in listdir(Videos)][0]
Files = listdir(Directory)
Video = path.abspath(path.join(Directory, [File for File in Files if File.endswith(".mp4")][0]))
Thumbnail = path.abspath(path.join(Directory, "Thumbnail.jpg"))
Archive_Name = path.basename(Directory) + ".zip"

#-=-=-=-#

if not Debug:
	# Archive Creation
	print(Styles.Reset + "\n\r" + "─" * 33 + "\n")
	print(("Making Archive" if Thumbnail else "Downloading Video") + "...")

	with ZipFile(Archive_Name, "w") as Archive:
		Archive.write(Video)
		Archive.write(Thumbnail)

	files.download(Archive_Name)
	print(Styles.OK + "Done!\n" + Styles.Reset)

	#-=-=-=-#

	# HTML Setup
	display(
		HTML(
			"""
			<video autoplay controls onloadstart="this.volume = 0.375" height=256>
				<source
					src="data:video/mp4;base64,{0}"
					type="video/mp4"
				>
			</video>
			<br><br><hr><br>
			""".format(
				b64encode(open(Video, "rb").read()).decode()
			)
		)
	)